In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
seq_len = 28
in_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 2
lr = 0.003

In [3]:
# MNIST Dataset
train_dataset = dsets.MNIST(root="./datasets/",
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=False)

test_dataset = dsets.MNIST(root="./datasets/",
                           train=False,
                           transform=transforms.ToTensor())

In [4]:
# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [8]:
# BiRNN (Many-to-one)
class BiRNN(nn.Module):
    def __init__(self, in_size, hidden_size, num_layers, num_classes):
        super(BiRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(in_size, hidden_size, num_layers,
                           batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)
        
    def forward(self, x):
        h0 = Variable(torch.zeros(self.num_layers*2, x.size(0), 
                                  self.hidden_size))
        c0 = Variable(torch.zeros(self.num_layers*2, x.size(0),
                                 self.hidden_size))
        
        out, _ = self.lstm(x, (h0, c0))
        
        out = self.fc(out[:,-1,:])
        return out

In [9]:
rnn = BiRNN(in_size, hidden_size, num_layers, num_classes)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)

In [11]:
# Train the Model 
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, seq_len, in_size))
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = rnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' 
%(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

Epoch [1/2], Step [100/600], Loss: 0.6022
Epoch [1/2], Step [200/600], Loss: 0.3746
Epoch [1/2], Step [300/600], Loss: 0.1474
Epoch [1/2], Step [400/600], Loss: 0.1325
Epoch [1/2], Step [500/600], Loss: 0.1848
Epoch [1/2], Step [600/600], Loss: 0.1574
Epoch [2/2], Step [100/600], Loss: 0.0213
Epoch [2/2], Step [200/600], Loss: 0.1653
Epoch [2/2], Step [300/600], Loss: 0.1069
Epoch [2/2], Step [400/600], Loss: 0.0552
Epoch [2/2], Step [500/600], Loss: 0.0208
Epoch [2/2], Step [600/600], Loss: 0.0488


In [13]:
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, seq_len, in_size))
    outputs = rnn(images)
    _, pred = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (pred == labels).sum()
    
print("Test Accuracy of the model on the 10000 test images: %d %%"
     % (100 * correct / total))

Test Accuracy of the model on the 10000 test images: 97 %


In [14]:
# save the model
# torch.save(rnn.state_dict(), "rnn.pkl")